## ResNet Original Training & Inference

#### Training ResNet50 with Cifar 10

In [1]:
PATH="checkpoints/resnet101_100.pth"

In [2]:
from ResNet.resnet import *
from ResNet.train import *

resnet_train_cifar10(device="cuda", blocks=101, PATH=PATH)

/home/work/m4ml-cc/DeepLearning-Study/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,472
       BatchNorm2d-2           [-1, 64, 16, 16]             128
         MaxPool2d-3             [-1, 64, 8, 8]               0
            Conv2d-4             [-1, 64, 8, 8]           4,096
       BatchNorm2d-5             [-1, 64, 8, 8]             128
            Conv2d-6             [-1, 64, 8, 8]          36,864
       BatchNorm2d-7             [-1, 64, 8, 8]             128
            Conv2d-8            [-1, 256, 8, 8]          16,384
       BatchNorm2d-9            [-1, 256, 8, 8]             512
           Conv2d-10            [-1, 256, 8, 8]          16,384
      BatchNorm2d-11            [-1, 256, 8, 8]             512
       BottleNeck-12            [-1, 256, 8, 8]               0
           Conv2d-13       

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# define dataset
cifar10_train = datasets.CIFAR10(root="./Data/", train=True, transform=transform, target_transform=None, download=True)
cifar10_test = datasets.CIFAR10(root="./Data/", train=False, transform=transform, target_transform=None, download=True)

# define loader
train_loader = DataLoader(cifar10_train,batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(cifar10_test,batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

model = torch.load(PATH)
model.eval()

test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.to('cuda')
        targets = targets.to('cuda')
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

acc = 100 * correct / total
print('test epoch : {} [{}/{}]| loss: {:.3f} | acc: {:.3f}'.format(
    epoch, batch_idx, len(test_loader), test_loss / (batch_idx + 1), acc))

writer.add_scalar('log/test error', 100 - acc, global_steps)

if acc > best_acc:
    print('==> Saving model..')
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir('save_model'):
        os.mkdir('save_model')
    torch.save(state, './save_model/ckpt.pth')
    best_acc = acc

return best_acc